# Cleaning Diagnosis file

Diagnosis file includes asthma, COPD, Bronchiectasis, acute exacerbation and adrenal insufficiency diagnosis records.

In [1]:
# load R packages
library(readxl)
packageVersion('readxl')
library(dplyr)
packageVersion('dplyr')
library(stringr)
packageVersion('stringr')
library(fastDummies)
packageVersion('fastDummies')
library(tidyr)
packageVersion('tidyr')
library(lubridate)
packageVersion('lubridate')
library(ggplot2)
packageVersion('ggplot2')

[1] ‘1.4.1’


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




[1] ‘1.1.0’

[1] ‘1.4.1’

[1] ‘1.6.3’

[1] ‘1.2.1’


Attaching package: ‘lubridate’


The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union




[1] ‘1.8.0’

[1] ‘3.4.4’

In [2]:
# set directory
project.dir = '...'
data.dir = '...'
regeps.dir = '...'
raw.rpdr.dir = file.path(regeps.dir, '...')
cleaned.rpdr.dir = file.path(regeps.dir, '...')
Qingwen.data.dir = file.path(regeps.dir, '...')

# Prepare data samples

In [3]:
# load medication file created by Yulu to extract plasma collect date
plasma.collect.dates <- read.csv(file.path(Qingwen.data.dir, 'med_corticosteroids_summary.csv'))
dim(plasma.collect.dates)
length(unique(plasma.collect.dates$EMPI)) # 928

[1] 1210  309

[1] 928

In [4]:
# Subject ID is biobank ID, do not use EMPI because of missing values
plasma.collect.dates <- plasma.collect.dates %>% select(Biobank_Subject_ID, COLLECTION_DATE)

# assign new names to the columns of the data frame
colnames(plasma.collect.dates) <- c('Subject_Id','Plasma_collect_date')
dim(plasma.collect.dates)
length(unique(plasma.collect.dates$Subject_Id))

[1] 1210    2

[1] 928

In [5]:
# remove duplicated rows
dim(plasma.collect.dates[duplicated(plasma.collect.dates), ]) # 275 duplicated rows
plasma.collect.dates.no.dup <- plasma.collect.dates[!duplicated(plasma.collect.dates), ]
dim(plasma.collect.dates.no.dup)

[1] 275   2

[1] 935   2

In [6]:
# remove duplicated Subject ID. Of 935, 7 are duplicated
plasma.collect.dates.no.dup$dup.ID <- duplicated(plasma.collect.dates.no.dup$Subject_Id)
table(plasma.collect.dates.no.dup$dup.ID) # 7 are duplicated ID

plasma.collect.dates.no.dup <- plasma.collect.dates.no.dup %>% filter(plasma.collect.dates.no.dup$dup.ID == 'FALSE')
dim(plasma.collect.dates.no.dup)


FALSE  TRUE 
  928     7 

[1] 928   3

In [7]:
# extract biobank ID
bib.data <- read.delim(file.path(raw.rpdr.dir, 'Bib.txt'), sep = '|')
biobank.ID <- bib.data %>% select(Subject_Id, EMPI)

In [9]:
# merge to have plasma collect date
data.id <- plasma.collect.dates.no.dup[,c('Subject_Id', 'Plasma_collect_date')] %>% 
                                        left_join(biobank.ID, by = 'Subject_Id')
dim(data.id)

[1] 928   3

# Diagnosis file

In [10]:
# check diagnosis file to see how many rows
diag.lines <-readLines(file.path(raw.rpdr.dir, 'Dia.txt'))
length(diag.lines) # 2618500 include headers

[1] 2618500

In [11]:
# load diganosis file
diag.data <- read.delim(file.path(raw.rpdr.dir, 'Dia.txt'), sep = '|')
dim(diag.data)

[1] 2618499      14

In [12]:
# merge biobank.ID and diagnosis file to have Subject_Id
diag.data.ID <- merge(diag.data, data.id[,c('Subject_Id', 'EMPI')], by = 'EMPI')
dim(diag.data.ID)

[1] 2618499      15

In [13]:
# format of date is character --> change format
diag.data.ID$Date <- as.Date(diag.data.ID$Date, format = '%m/%d/%Y')
typeof(diag.data.ID$Date)

[1] "double"

# Asthma

- Asthma diagnosis and category are downloaded from Biobank Portal: asthma_diagnosis.csv
- Filter asthma diagnosis in RPDR file by searching asthma or Asthma in Diagnosis_Name column
- Compare diagnosis name in RPDR file with Biobank Portal
- Some asthma diagnosis do not have category info in Biobank Portal (RPDR does not have asthma category). No Category can replaced to Other and Unspecififed Asthma

In [14]:
# load list of asthma categorgy in asthma folder from Biobank portal
asth.diag.biobank <- read.csv(file.path(data.dir, 'asthma_diagnosis.csv'))

# unique diagnosis name in Biobank Portal
diganosis.from.biobank <- unique(asth.diag.biobank$Diagnosis_Name)

# subset data with asthma or Asthma in Diganosis_Names
asth.diag <- diag.data.ID %>% filter(str_detect(Diagnosis_Name, 'Asthma|asthma'))
diganosis.from.rpdr <- unique(asth.diag$Diagnosis_Name)

In [15]:
# diagnosis not in rpdr
setdiff(diganosis.from.biobank, diganosis.from.rpdr)

[1] "Extrinsic asthma"                              
[2] "Intrinsic asthma"                              
[3] "Mild persistent asthma with status asthmaticus"
[4] "Asthma, acute exacerbation-LMR 1288"           
[5] "Exercise induced bronchospasm"                 
[6] "Exercise-induced asthma-LMR 1586"              
[7] "Other forms of asthma"

In [16]:
# diagnosis not in biobank
setdiff(diganosis.from.rpdr, diganosis.from.biobank)

[1] "Bronchitis and asthma age >17 w/o CC (AP v21)"                                         
 [2] "Bronchitis And Asthma Age >17 W/O Cc (MS v24)"                                         
 [3] "Chronic obstructive asthma with acute exacerbation"                                    
 [4] "Bronchitis And Asthma Age >17 W Cc (MS v24)"                                           
 [5] "Chronic obstructive asthma, without mention of status asthmaticus"                     
 [6] "Bronchitis and asthma age >17 w CC (AP v21)"                                           
 [7] "Family history of asthma"                                                              
 [8] "Chronic obstructive asthma, with status asthmaticus"                                   
 [9] "Family history of asthma and other chronic lower respiratory diseases"                 
[10] "Asthma (APR v30)"                                                                      
[11] "Simple pneumonia and oth respiratory disord exc bronchitis, asthma w major cc (AP v21)"
[12] "Poisoning by antiasthmatics"                                                           
[13] "Bronchitis And Asthma Age 0-17 (MS v24)"                                               
[14] "Bronchitis and asthma age <18 w/o CC (AP v21)"                                         
[15] "Eosinophilic asthma"                                                                   
[16] "Adverse effect of antiasthmatics, initial encounter"

#### Note:
- Above asthma diagnosis are not in asthma folder in Biobank. For example, some are classifed in COPD category in Biobank. They are **Chronic obstructive asthma, without mention of status asthmaticus, Chronic obstructive asthma with acute exacerbation, Asthma (APR v30), Chronic obstructive asthma, with status asthmaticus, Eosinophilic asthma**
- Bronchitis and asthma diagnosis are in Bronchitis and asthma (Disease related group), not asthma folder in Biobank

-> **use asthma diagnosis name in Biobank Portal to find asthma diagnosis in RPDR file**

In [17]:
# filter asthma
asth.diag <- filter(diag.data.ID, Diagnosis_Name %in% asth.diag.biobank$Diagnosis_Name) 
dim(asth.diag)
length(unique(asth.diag$Subject_Id)) # all 928 patients have asthma

[1] 64098    15

[1] 928

In [18]:
# select columns
asth.diag <- asth.diag %>% select(Subject_Id, Date, Diagnosis_Name, Code_Type, Code,  
                                  Diagnosis_Flag, Inpatient_Outpatient, Provider, 
                                  Hospital, Encounter_number)
dim(asth.diag)

[1] 64098    10

In [19]:
dim(asth.diag[duplicated(asth.diag), ]) # check duplication: 497

[1] 497  10

In [20]:
# remove duplicated rows
asth.diag.no.dup <- asth.diag[!duplicated(asth.diag), ]
dim(asth.diag.no.dup) # remove 497

[1] 63601    10

In [21]:
# match with biobank file to have asthma sub category
asth.diag.no.dup <- asth.diag.no.dup %>% left_join(asth.diag.biobank[,c('Diagnosis_Name', 'Asthma_Category')], 
                                                  by = 'Diagnosis_Name') # remove biobank path
dim(asth.diag.no.dup)
table(asth.diag.no.dup$Asthma_Category)

[1] 63601    11


     Mild_intermittent_asthma        Mild_persistent_asthma 
                        12280                          2356 
   Moderate_persistent_asthma                   No_category 
                         6989                           773 
Other_and_unspecificed_asthma      Severe_persistent_asthma 
                        35651                          5552 

In [22]:
# change No category to Other_and_unspecificed_asthma
asth.diag.no.dup$Asthma_Category[asth.diag.no.dup$Asthma_Category == 'No_category'] <- 'Other_and_unspecificed_asthma'
table(asth.diag.no.dup$Asthma_Category)
sum(is.na(asth.diag.no.dup$Asthma_Category)) # check missing


     Mild_intermittent_asthma        Mild_persistent_asthma 
                        12280                          2356 
   Moderate_persistent_asthma Other_and_unspecificed_asthma 
                         6989                         36424 
     Severe_persistent_asthma 
                         5552 

[1] 0

## Asthma: Remove duplication

In [23]:
# check duplication
dim(asth.diag.no.dup[duplicated(asth.diag.no.dup[,c('Subject_Id', 'Date', 'Asthma_Category', 'Encounter_number')]), ]) # 9314
dim(asth.diag.no.dup[duplicated(asth.diag.no.dup[,c('Subject_Id', 'Date', 'Asthma_Category')]), ]) # 26785

[1] 9314   11

[1] 26785    11

Because some patients have different diagnosis at the same date, select the more severity asthma. 
The rank will be **other and unspecified < mild intermittent < mild persistent < severe**

In [32]:
# rank based on severity
table(asth.diag.no.dup$Asthma_Category)
asth.diag.no.dup$Cat_prior <- NA
asth.diag.no.dup$Cat_prior[asth.diag.no.dup$Asthma_Category == 'Other_and_unspecificed_asthma'] <- 0
asth.diag.no.dup$Cat_prior[asth.diag.no.dup$Asthma_Category == 'Mild_intermittent_asthma'] <- 1
asth.diag.no.dup$Cat_prior[asth.diag.no.dup$Asthma_Category == 'Mild_persistent_asthma'] <- 2
asth.diag.no.dup$Cat_prior[asth.diag.no.dup$Asthma_Category == 'Moderate_persistent_asthma'] <- 3
asth.diag.no.dup$Cat_prior[asth.diag.no.dup$Asthma_Category == 'Severe_persistent_asthma'] <- 4
typeof(asth.diag.no.dup$Cat_prior)
table(asth.diag.no.dup$Cat_prior)


     Mild_intermittent_asthma        Mild_persistent_asthma 
                        12280                          2356 
   Moderate_persistent_asthma Other_and_unspecificed_asthma 
                         6989                         36424 
     Severe_persistent_asthma 
                         5552 

[1] "double"


    0     1     2     3     4 
36424 12280  2356  6989  5552 

In [33]:
# priority selecting patients with asthma severity
asth.cat.prior <- asth.diag.no.dup %>%                                 # Get max by group
  group_by(Subject_Id, Date) %>%
  summarise_at(vars(Cat_prior),
               list(Cat_prior = max))
dim(asth.cat.prior)

[1] 34716     3

In [34]:
dim(asth.cat.prior[duplicated(asth.cat.prior[,c('Subject_Id', 'Date')]), ]) # no dup

[1] 0 3

In [35]:
# change back to Asthma Category
table(asth.cat.prior$Cat_prior)
asth.cat.prior$Asthma_Category <- NA
asth.cat.prior$Asthma_Category[asth.cat.prior$Cat_prior == 0] <- 'Other_and_unspecificed_asthma'
asth.cat.prior$Asthma_Category[asth.cat.prior$Cat_prior == 1] <- 'Mild_intermittent_asthma'
asth.cat.prior$Asthma_Category[asth.cat.prior$Cat_prior == 2] <- 'Mild_persistent_asthma'
asth.cat.prior$Asthma_Category[asth.cat.prior$Cat_prior == 3] <- 'Moderate_persistent_asthma'
asth.cat.prior$Asthma_Category[asth.cat.prior$Cat_prior == 4] <- 'Severe_persistent_asthma'
table(asth.cat.prior$Cat_prior)
dim(asth.cat.prior)


    0     1     2     3     4 
21146  7065  1211  3188  2106 


    0     1     2     3     4 
21146  7065  1211  3188  2106 

[1] 34716     4

In [36]:
# export data with unique category for each date
asth.cat.prior <- asth.cat.prior %>% select(Subject_Id, Date, Asthma_Category)

In [37]:
# Total diagnosis record for each patient
asth.cat.prior$Diagnosis <- 1 # assign 1 for asthma diagnosis

# Group by Subject_Id and sum of diagnosis date using dplyr
asth.counts.per.ind <- asth.cat.prior %>% group_by(Subject_Id) %>% 
  summarise(Total_Asthma_Diagnosis = sum(Diagnosis),
            .groups = 'drop')
dim(asth.counts.per.ind)

[1] 928   2

In [38]:
# summary statistic
summary(asth.counts.per.ind$Total_Asthma_Diagnosis)
table(asth.counts.per.ind$Total_Asthma_Diagnosis)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
   1.00   10.00   21.00   37.41   44.25  333.00 


  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18  19  20 
 12  25  20  14  29  22  26  29  34  30  32  32  23  21  17  18  17  25  17  17 
 21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36  37  38  39  40 
 21  10  20  17   6  19   8  14  11  10   4  10  11   7   8   8   9   4   7   4 
 41  42  43  44  45  46  47  48  49  50  51  52  53  54  55  56  57  58  59  60 
  9   8   7   4   4   6   6   7   8   7   3   3   4   3   6   4   6   4   3   2 
 61  62  63  64  65  66  67  68  69  70  71  72  73  74  75  76  77  78  79  80 
  5   2   3   1   2   1   2   4   3   2   1   2   1   5   3   3   3   1   3   3 
 81  82  83  85  86  87  89  90  92  93  94  95  96  97  98  99 100 101 102 103 
  3   4   3   1   1   1   3   1   1   1   1   2   2   3   1   2   1   1   1   2 
106 107 110 111 112 113 115 117 118 121 124 126 127 128 129 132 135 136 140 141 
  1   2   3   2   1   1   1   2   1   1   2   1   1   2   1   2   1   2   1   1 
142 143 147 151 155 157 159

## Asthma: Most frequency diagnosis
- Find diagnosis have the most number of record for each patient

In [39]:
# create dummy variables for each diag category in asth.cat.prior
asthma.data.dummy <- dummy_cols(asth.cat.prior,
                   select_columns = 'Asthma_Category')
dim(asthma.data.dummy)

[1] 34716     9

In [40]:
which(colnames(asthma.data.dummy) == 'Asthma_Category_Mild_intermittent_asthma')

[1] 5

In [41]:
# group by Subject_Id and sum all columns
asthma.data.dummy.group <- asthma.data.dummy[,-2:-4] # remove un-neccesary columns
sum.asth.diganosis <- asthma.data.dummy.group %>% group_by(Subject_Id) %>% 
                                                    summarise(across(everything(), sum), .groups = 'drop') %>%
                                                    as.data.frame()
dim(sum.asth.diganosis)

[1] 928   6

In [42]:
# add column: Yes or No for each diagnosis
exist.sum.asth.diganosis <- sum.asth.diganosis
exist.sum.asth.diganosis[,-1] <- ifelse(exist.sum.asth.diganosis[,-1] > 0, 'Yes', 'No')
colnames(exist.sum.asth.diganosis)[-1] <- paste(colnames(exist.sum.asth.diganosis[,-1]),'_Existence_Yes_No', sep = '')

In [43]:
# add suffix count after each diagnosis
colnames(sum.asth.diganosis)<- paste(colnames(sum.asth.diganosis),'total_diagnosis',sep='_')
colnames(sum.asth.diganosis)[1] <- 'Subject_Id' 

In [44]:
# merge
sum.asth.diganosis.final <- merge(sum.asth.diganosis, exist.sum.asth.diganosis, by = 'Subject_Id')
dim(sum.asth.diganosis.final)

[1] 928  11

In [45]:
# Remove the 'Asthma_Category_' prefix from all column names
names(sum.asth.diganosis.final) <- gsub('Asthma_Category_', '', names(sum.asth.diganosis.final))
dim(sum.asth.diganosis.final)

[1] 928  11

In [46]:
# add column: Yes or No for asthma diagnosis total
sum.asth.diganosis.final$Any_Asthma_Diagnosis_Existence_Yes_No <- 'Yes' # yes for asthma diagnosis

In [47]:
# find most frequency asthma diagnosis
asthma.cat.count <- sum.asth.diganosis.final[,colnames(sum.asth.diganosis.final[,c(2:6)])]

sum.asth.diganosis.final$Most_freq_asthma_diagnosis <- colnames(asthma.cat.count)[apply(asthma.cat.count,1,which.max)]

# remove suffix _total_diagnosis
sum.asth.diganosis.final <- sum.asth.diganosis.final %>% mutate_at('Most_freq_asthma_diagnosis', str_replace, '_total_diagnosis', '')

# relocate 
sum.asth.diganosis.final <- sum.asth.diganosis.final %>% relocate(Most_freq_asthma_diagnosis, .after = Any_Asthma_Diagnosis_Existence_Yes_No)

In [48]:
table(sum.asth.diganosis.final$Most_freq_asthma_diagnosis)


     Mild_intermittent_asthma        Mild_persistent_asthma 
                          134                            19 
   Moderate_persistent_asthma Other_and_unspecificed_asthma 
                           38                           712 
     Severe_persistent_asthma 
                           25 

## Asthma: Closest diagnosis before or after plasma collect date

In [49]:
# merge plasma collect date by Subject_ID
asth.cat.prior.1 <- merge(asth.cat.prior, data.id[,c('Subject_Id', 'Plasma_collect_date')], 
                          by = 'Subject_Id')
dim(asth.cat.prior.1)

[1] 34716     5

In [50]:
# convert the date column to the Y-M-D format
asth.cat.prior.1$Date <- as.Date(asth.cat.prior.1$Date, format = '%m/%d/%Y')
typeof(asth.cat.prior.1$Date)

# plasma collect date
asth.cat.prior.1$Plasma_collect_date <- as.Date(asth.cat.prior.1$Plasma_collect_date, format = '%Y -%m -%d')
typeof(asth.cat.prior.1$Plasma_collect_date)

[1] "double"

[1] "double"

In [51]:
# substract collect date and diag date
asth.cat.prior.1['Days_Difference'] <- difftime(asth.cat.prior.1$Plasma_collect_date, 
                                                asth.cat.prior.1$Date, units = 'days')
head(asth.cat.prior.1$Days_Difference)
typeof(asth.cat.prior.1$Days_Difference)

# absolute values because of including before and after plasma collect date
asth.cat.prior.1['Days_Difference_Abs'] <- as.numeric(abs(asth.cat.prior.1$Days_Difference))
typeof(asth.cat.prior.1$Days_Difference_Abs)

Time differences in days
[1]  3273  3059  2623  1700  -435 -1419

[1] "double"

[1] "double"

In [52]:
dim(asth.cat.prior.1[duplicated(asth.cat.prior.1), ]) # no dup

[1] 0 7

In [53]:
# select intersted columns
diag.date.closest.collect <- asth.cat.prior.1 %>% select(Subject_Id, Date, Days_Difference_Abs, Asthma_Category)

In [54]:
# find closest date to plasma collect date in diag.date.closest.collect
closest.collect.date <- diag.date.closest.collect %>%         # Get min by group because of positive value
  group_by(Subject_Id, Asthma_Category) %>%
  summarise_at(vars(Days_Difference_Abs),
               list(Closest_collect_date_gap = min))
dim(closest.collect.date)

[1] 2268    3

In [55]:
dim(closest.collect.date[duplicated(closest.collect.date), ]) # no dup

[1] 0 3

In [56]:
# group the data frame by 'Subject_Id' and spread the 'closet_collect_date' column
closest.collect.date.wide <- closest.collect.date %>% group_by(Subject_Id, Asthma_Category) %>%
                                                      pivot_wider(names_from = Asthma_Category, 
                                                                  values_from = Closest_collect_date_gap)
dim(closest.collect.date.wide)

[1] 928   6

In [57]:
# add suffix cloest collect date after each diag
colnames(closest.collect.date.wide) <- paste(colnames(closest.collect.date.wide),'closest_collect_date_gap',sep='_')
colnames(closest.collect.date.wide)[1] = 'Subject_Id'
dim(closest.collect.date.wide)

[1] 928   6

In [58]:
# filter for all diagnosis to include diagnosis date for each days difference
diag.names <- unique(diag.date.closest.collect$Asthma_Category)
for (diag.name in diag.names){
    diag.date.closest.collect.each.diag <- diag.date.closest.collect %>% filter(Asthma_Category == diag.name)


    # remove diagnosis_name and Plasma_collect_date
    diag.date.closest.collect.each.diag.1 <- diag.date.closest.collect.each.diag %>% select(-c(Asthma_Category))


    # replace Date name with date + diagnosis
    names(diag.date.closest.collect.each.diag.1)[names(diag.date.closest.collect.each.diag.1) == 'Date'] <- paste(diag.name, '_closest_collect_date', sep = '')


    # test merge date for diagnosis
    closest.collect.date.wide <- merge(closest.collect.date.wide, diag.date.closest.collect.each.diag.1, 
                                                        by.x = c('Subject_Id', paste(diag.name, '_closest_collect_date_gap', sep = '')), 
                                                        by.y = c('Subject_Id', 'Days_Difference_Abs'), all.x = TRUE)
    }


dim(closest.collect.date.wide)

[1] 933  11

In [59]:
# Duplication because of absoluted days difference -> remove duplication
check_unique_list <- c('Subject_Id',
                       'Severe_persistent_asthma_closest_collect_date_gap',
                       'Moderate_persistent_asthma_closest_collect_date_gap',
                       'Mild_persistent_asthma_closest_collect_date_gap',
                       'Other_and_unspecificed_asthma_closest_collect_date_gap',
                       'Mild_intermittent_asthma_closest_collect_date_gap')
closest.collect.date.wide <- closest.collect.date.wide[!duplicated(closest.collect.date.wide[,check_unique_list]), ]
dim(closest.collect.date.wide)

[1] 928  11

In [60]:
# find closest asthma diagnosis to collect date
asthma.closest.collect.date.cols <- closest.collect.date.wide[,c('Mild_intermittent_asthma_closest_collect_date_gap', 
                                                'Mild_persistent_asthma_closest_collect_date_gap',
                                                'Moderate_persistent_asthma_closest_collect_date_gap',
                                                'Severe_persistent_asthma_closest_collect_date_gap',
                                                'Other_and_unspecificed_asthma_closest_collect_date_gap')]


closest.collect.date.wide$Closest_collect_date_asthma_diagnosis <- colnames(asthma.closest.collect.date.cols)[apply(data.matrix(asthma.closest.collect.date.cols),1,which.min)]

# remove suffix _closest_collect_date_gap
closest.collect.date.wide <- closest.collect.date.wide %>% mutate_at('Closest_collect_date_asthma_diagnosis', str_replace, '_closest_collect_date_gap', '')

In [61]:
table(closest.collect.date.wide$Closest_collect_date_asthma_diagnosis)


     Mild_intermittent_asthma        Mild_persistent_asthma 
                          164                            30 
   Moderate_persistent_asthma Other_and_unspecificed_asthma 
                           52                           663 
     Severe_persistent_asthma 
                           19 

In [62]:
# gap between closest date to collect date
min.na.rm <- function(x){
    min(x, na.rm = TRUE)
}

closest.collect.date.wide$Closest_collect_date_asthma_diagnosis_gap <- apply(data.matrix(asthma.closest.collect.date.cols),1,min.na.rm)
# relocate 
closest.collect.date.wide <- closest.collect.date.wide %>% relocate(Closest_collect_date_asthma_diagnosis_gap, .after = Closest_collect_date_asthma_diagnosis)

## Asthma: Recent date

In [63]:
typeof(asth.cat.prior.1$Date) # make sure date format is not character

[1] "double"

In [64]:
# find recent date in asth.cat.prior.1
recent.asth.date <- asth.cat.prior.1 %>%                                       # Get max by group
  group_by(Subject_Id, Asthma_Category) %>%
  summarise_at(vars(Date),
               list(Recent_date = max))
dim(recent.asth.date)

[1] 2268    3

In [65]:
# group the data frame by 'Subject_Id' and spread the 'max' column
recent.asth.date.wide <- recent.asth.date %>% group_by(Subject_Id, Asthma_Category) %>%
                                                    pivot_wider(names_from = Asthma_Category, 
                                                                values_from = Recent_date)
dim(recent.asth.date.wide)

[1] 928   6

In [66]:
# add suffix recent date after each diagnosis
colnames(recent.asth.date.wide) <- paste(colnames(recent.asth.date.wide),'recent_diagnosis_date',sep='_')
colnames(recent.asth.date.wide)[1] = 'Subject_Id'

In [67]:
# find recent asthma diagnosis
# because recent date is string -> need to convert date formate
asthma.recent.date.cols <- recent.asth.date.wide[,c('Mild_intermittent_asthma_recent_diagnosis_date', 
                                                'Mild_persistent_asthma_recent_diagnosis_date',
                                                'Moderate_persistent_asthma_recent_diagnosis_date',
                                                'Severe_persistent_asthma_recent_diagnosis_date',
                                                'Other_and_unspecificed_asthma_recent_diagnosis_date')]

convert.date <- function(x) as.Date(x, format = '%Y-%m-%d')
asthma.recent.date.cols <- data.frame(lapply(asthma.recent.date.cols, convert.date))
asthma.recent.date.cols[is.na(asthma.recent.date.cols)] <- as.Date('1900-01-01', format = '%Y-%m-%d')
recent.asth.date.wide$Most_recent_asthma_diagnosis <- colnames(asthma.recent.date.cols)[apply(data.matrix(asthma.recent.date.cols),1,which.max)]

# remove suffix _recent_diagnosis_date
recent.asth.date.wide <- recent.asth.date.wide %>% mutate_at('Most_recent_asthma_diagnosis', str_replace, '_recent_diagnosis_date', '')

## Asthma: First date

In [68]:
# find first date in asth.cat.prior.1
first.asth.date <- asth.cat.prior.1  %>%                                      # Get min by group
  group_by(Subject_Id, Asthma_Category) %>%
  summarise_at(vars(Date),
               list(First_date = min))
dim(first.asth.date)

[1] 2268    3

In [69]:
# group the data frame by 'Subject_Id' and spread the 'min' column
first.asth.date.wide <- first.asth.date %>% group_by(Subject_Id, Asthma_Category) %>%
                                                    pivot_wider(names_from = Asthma_Category, 
                                                                values_from = First_date)
dim(first.asth.date.wide)

[1] 928   6

In [70]:
# add suffix first date after each diagnosis
colnames(first.asth.date.wide) <- paste(colnames(first.asth.date.wide),'first_diagnosis_date',sep='_')
colnames(first.asth.date.wide)[1] = 'Subject_Id'

In [71]:
# merge files
dim(closest.collect.date.wide)
dim(recent.asth.date.wide)
dim(asth.counts.per.ind)
dim(first.asth.date.wide)
dim(sum.asth.diganosis.final)

[1] 928  13

[1] 928   7

[1] 928   2

[1] 928   6

[1] 928  13

In [72]:
asthma.final <- sum.asth.diganosis.final %>% left_join(asth.counts.per.ind, by = 'Subject_Id') %>%
                                             left_join(closest.collect.date.wide, by = 'Subject_Id') %>%
                                             left_join(recent.asth.date.wide, by = 'Subject_Id') %>%
                                             left_join(first.asth.date.wide, by = 'Subject_Id')
dim(asthma.final)

[1] 928  37

In [73]:
asthma.final <- asthma.final %>% select(Subject_Id,
                                        Any_Asthma_Diagnosis_Existence_Yes_No,
                                        Total_Asthma_Diagnosis,
                                        Most_freq_asthma_diagnosis,
                                        Most_recent_asthma_diagnosis,
                                        Closest_collect_date_asthma_diagnosis,
                                        Closest_collect_date_asthma_diagnosis_gap,
                                        
                                        Mild_intermittent_asthma_Existence_Yes_No,
                                        Mild_intermittent_asthma_total_diagnosis,
                                        
                                        Mild_intermittent_asthma_first_diagnosis_date,
                                        Mild_intermittent_asthma_recent_diagnosis_date,
                                        Mild_intermittent_asthma_closest_collect_date,
                                        Mild_intermittent_asthma_closest_collect_date_gap,
                                        Mild_persistent_asthma_Existence_Yes_No,
                                        Mild_persistent_asthma_total_diagnosis,
                                        
                                        Mild_persistent_asthma_first_diagnosis_date,
                                        Mild_persistent_asthma_recent_diagnosis_date,
                                        Mild_persistent_asthma_closest_collect_date,
                                        Mild_persistent_asthma_closest_collect_date_gap,
                                        Moderate_persistent_asthma_Existence_Yes_No,
                                        Moderate_persistent_asthma_total_diagnosis,
                                        
                                        Moderate_persistent_asthma_first_diagnosis_date,
                                        Moderate_persistent_asthma_recent_diagnosis_date,
                                        Moderate_persistent_asthma_closest_collect_date,
                                        Moderate_persistent_asthma_closest_collect_date_gap,
                                        Severe_persistent_asthma_Existence_Yes_No,
                                        Severe_persistent_asthma_total_diagnosis,
                                        
                                        Severe_persistent_asthma_first_diagnosis_date,
                                        Severe_persistent_asthma_recent_diagnosis_date,
                                        Severe_persistent_asthma_closest_collect_date,
                                        Severe_persistent_asthma_closest_collect_date_gap,
                                        Other_and_unspecificed_asthma_Existence_Yes_No,
                                        Other_and_unspecificed_asthma_total_diagnosis,
                                        
                                        Other_and_unspecificed_asthma_first_diagnosis_date,
                                        Other_and_unspecificed_asthma_recent_diagnosis_date,
                                        Other_and_unspecificed_asthma_closest_collect_date,
                                        Other_and_unspecificed_asthma_closest_collect_date_gap)
dim(asthma.final)

[1] 928  37

# Bronchiectasis

In [74]:
# check Bronchiectasis based on the folder: Bronchiectasis in biobank
bron.list <- c('Bronchiectasis', 'Bronchiectasis with (acute) exacerbation', 'Bronchiectasis with acute exacerbation',
               'Bronchiectasis with acute lower respiratory infection', 'Bronchiectasis, uncomplicated', 
               'Bronchiectasis without acute exacerbation', 'Bronchiectasis-LMR 54', 'Bronchiectasis-Oncall')

In [75]:
# filter data
bron.diag <- diag.data.ID %>% filter(str_detect(Diagnosis_Name, 'Bronchiectasis|bronchiectasis'))
unique(bron.diag$Diagnosis_Name)

[1] "Bronchiectasis, uncomplicated"                        
 [2] "Bronchiectasis without acute exacerbation"            
 [3] "Bronchiectasis with (acute) exacerbation"             
 [4] "Bronchiectasis with acute lower respiratory infection"
 [5] "Bronchiectasis"                                       
 [6] "Bronchiectasis-LMR 54"                                
 [7] "Bronchiectasis with acute exacerbation"               
 [8] "Bronchiectasis-Oncall"                                
 [9] "Tuberculous bronchiectasis, unspecified examination"  
[10] "Congenital bronchiectasis"

In [76]:
# check seven not in the RPDR
diganosis.from.rpdr <- unique(bron.diag$Diagnosis_Name)

In [77]:
# not in rpdr
setdiff(bron.list, diganosis.from.rpdr) # everything in RPDR

character(0)

In [78]:
# not in biobank
setdiff(diganosis.from.rpdr, bron.list)

[1] "Tuberculous bronchiectasis, unspecified examination"
[2] "Congenital bronchiectasis"

- Tuberculous bronchiectasis, unspecified examination is classified in Tuberculous (Biobank portal)
- Congenital bronchiectasis is classified in Congenital malformation (Biobank portal)

In [79]:
# select cols
bron.diag <- bron.diag %>% select(Subject_Id, Date, Diagnosis_Name, Code_Type, Code,  
                                  Diagnosis_Flag, Inpatient_Outpatient, Provider, 
                                  Hospital, Encounter_number)
dim(bron.diag)

[1] 11939    10

In [80]:
dim(bron.diag[duplicated(bron.diag), ]) # check dup: 141

[1] 141  10

In [81]:
# remove duplicated rows
bron.diag.no.dup <- bron.diag[!duplicated(bron.diag), ]
dim(bron.diag.no.dup)

[1] 11798    10

In [82]:
dim(bron.diag.no.dup[duplicated(bron.diag.no.dup[,c('Subject_Id', 'Date')]), ]) # 5779

[1] 5779   10

In [83]:
unique(bron.diag.no.dup$Diagnosis_Name)

[1] "Bronchiectasis, uncomplicated"                        
 [2] "Bronchiectasis without acute exacerbation"            
 [3] "Bronchiectasis with (acute) exacerbation"             
 [4] "Bronchiectasis with acute lower respiratory infection"
 [5] "Bronchiectasis"                                       
 [6] "Bronchiectasis-LMR 54"                                
 [7] "Bronchiectasis with acute exacerbation"               
 [8] "Bronchiectasis-Oncall"                                
 [9] "Tuberculous bronchiectasis, unspecified examination"  
[10] "Congenital bronchiectasis"

In [84]:
# Bronchiectasis with (acute) exacerbation = Bronchiectasis with acute exacerbation.
bron.diag.no.dup$Diagnosis_Name[bron.diag.no.dup$Diagnosis_Name == 'Bronchiectasis with (acute) exacerbation'] <- 'Bronchiectasis with acute exacerbation'

In [85]:
dim(bron.diag.no.dup[duplicated(bron.diag.no.dup[,c('Subject_Id','Diagnosis_Name', 'Date')]), ]) # dup same diag

[1] 5287   10

In [86]:
bron.diag.no.dup.1 <- bron.diag.no.dup[!duplicated(bron.diag.no.dup[,c('Subject_Id','Diagnosis_Name', 'Date')]), ]
dim(bron.diag.no.dup.1)

[1] 6511   10

In [88]:
# because we just want to check patient have bronchectasis or not so just remove duplicated date
bron.diag.no.dup.2 <- bron.diag.no.dup.1[!duplicated(bron.diag.no.dup.1[,c('Subject_Id','Date')]), ] 
dim(bron.diag.no.dup.2)
bron.diag.no.dup.2[duplicated(bron.diag.no.dup.2[,c('Subject_Id','Date')]), ] # no dup

[1] 6019   10

Subject_Id,Date,Diagnosis_Name,Code_Type,Code,Diagnosis_Flag,Inpatient_Outpatient,Provider,Hospital,Encounter_number
<int>,<date>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>


In [89]:
bron.diag.no.dup.2$Diagnosis <- 1 # assign 1 for bronc diagnosis

In [90]:
# Group by Subject_Id and sum of diagnosis
bronc.counts.per.ind <- bron.diag.no.dup.2 %>% group_by(Subject_Id) %>% 
  summarise(Total_Bronchiectasis_Diagnosis = sum(Diagnosis),
            .groups = 'drop')
dim(bronc.counts.per.ind)

[1] 194   2

In [91]:
# add column: Yes or No for bronc.data diagnosis total
bronc.counts.per.ind$Any_Bronchiectasis_Existence_Yes_No <- 'Yes' # yes for diagnosis

In [92]:
# final flie: bronc.counts.per.ind
bronc.final <- bronc.counts.per.ind

# Chronic Bronchitis

In [93]:
# filter data
bronchitis.diag <- diag.data.ID %>% filter(str_detect(Diagnosis_Name, 'Chronic bronchitis|chronic bronchitis'))
unique(bronchitis.diag$Diagnosis_Name)

[1] "Obstructive chronic bronchitis, without mention of acute exacerbation"
 [2] "Obstructive chronic bronchitis with acute exacerbation"               
 [3] "Mucopurulent chronic bronchitis"                                      
 [4] "Obstructive chronic bronchitis with acute bronchitis"                 
 [5] "Unspecified chronic bronchitis"                                       
 [6] "Mixed simple and mucopurulent chronic bronchitis"                     
 [7] "Simple chronic bronchitis"                                            
 [8] "Other chronic bronchitis"                                             
 [9] "Obstructive chronic bronchitis"                                       
[10] "Chronic bronchitis-Oncall"                                            
[11] "Chronic bronchitis"

In [94]:
length(unique(bronchitis.diag$Subject_Id))

[1] 196

In [95]:
dim(bronchitis.diag[duplicated(bronchitis.diag[,c('Subject_Id','Diagnosis_Name', 'Date')]), ])

[1] 1093   15

In [96]:
# because we just want to check patient have bronchectasis or not so just remove duplicated date
bronchitis.diag.no.dup <- bronchitis.diag[!duplicated(bronchitis.diag[,c('Subject_Id','Date')]), ] 
dim(bronchitis.diag.no.dup)
bronchitis.diag.no.dup[duplicated(bronchitis.diag.no.dup[,c('Subject_Id','Date')]), ] # no dup

[1] 1536   15

EMPI,EPIC_PMRN,MRN_Type,MRN,Date,Diagnosis_Name,Code_Type,Code,Diagnosis_Flag,Provider,Clinic,Hospital,Inpatient_Outpatient,Encounter_number,Subject_Id
<int>,<dbl>,<chr>,<int>,<date>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>


In [97]:
length(unique(bronchitis.diag.no.dup$Subject_Id))

[1] 196

In [98]:
bronchitis.diag.no.dup$Diagnosis <- 1 # assign 1 for bronc diagnosis

In [99]:
# Group by Subject_Id and sum of diagnosis
bronchtitis.counts.per.ind <- bronchitis.diag.no.dup %>% group_by(Subject_Id) %>% 
  summarise(Total_Chronic_Bronchitis_Diagnosis = sum(Diagnosis),
            .groups = 'drop')
dim(bronchtitis.counts.per.ind)

[1] 196   2

In [100]:
# add column: Yes or No for bronc.data diagnosis total
bronchtitis.counts.per.ind$Any_Chronic_Bronchitis_Existence_Yes_No <- 'Yes' # yes for diagnosis

In [101]:
bronchtitis.final <- bronchtitis.counts.per.ind

# Acute exacerbation

In [102]:
# Acute exacerbation variables under 'Chronic lower respiratory diseases' folder in Biobank
acu.exa.list <- c('Asthma, acute exacerbation-LMR 1288',
                             'Mild intermittent asthma with (acute) exacerbation', 
                             'Mild persistent asthma with (acute) exacerbation', 
                             'Moderate persistent asthma with (acute) exacerbation', 
                             'Asthma, unspecified type, with acute exacerbation', 
                             'Unspecified asthma with (acute) exacerbation',
                             'Severe persistent asthma with (acute) exacerbation', 
                             'Extrinsic asthma with acute exacerbation', 
                             'Intrinsic asthma, with acute exacerbation',
                             'Chronic obstructive asthma with acute exacerbation', 
                             'Obstructive chronic bronchitis with acute exacerbation')

In [103]:
# filter acute exacerbation related to asthma
exar.patterns <- c('Asthma|asthma', 'acute|Acute', 'exacerbation|Exacerbation')
exar.diag <- diag.data.ID %>% filter(str_detect(Diagnosis_Name, exar.patterns[1]) & 
                                    str_detect(Diagnosis_Name, exar.patterns[2]) &
                                    str_detect(Diagnosis_Name, exar.patterns[3]))
unique(exar.diag$Diagnosis_Name)

[1] "Severe persistent asthma with (acute) exacerbation"  
[2] "Asthma, unspecified type, with acute exacerbation"   
[3] "Unspecified asthma with (acute) exacerbation"        
[4] "Chronic obstructive asthma with acute exacerbation"  
[5] "Intrinsic asthma, with acute exacerbation"           
[6] "Extrinsic asthma with acute exacerbation"            
[7] "Mild persistent asthma with (acute) exacerbation"    
[8] "Mild intermittent asthma with (acute) exacerbation"  
[9] "Moderate persistent asthma with (acute) exacerbation"

In [104]:
# not in rpdr
setdiff(acu.exa.list, exar.diag$Diagnosis_Name)

[1] "Asthma, acute exacerbation-LMR 1288"                   
[2] "Obstructive chronic bronchitis with acute exacerbation"

In [105]:
# not in biobank
setdiff(exar.diag$Diagnosis_Name, acu.exa.list)

character(0)

In [106]:
# filter data in acute exacerbation list
exar.diag <- filter(diag.data.ID, Diagnosis_Name %in% acu.exa.list)
unique(exar.diag$Diagnosis_Name)

[1] "Severe persistent asthma with (acute) exacerbation"    
 [2] "Asthma, unspecified type, with acute exacerbation"     
 [3] "Unspecified asthma with (acute) exacerbation"          
 [4] "Chronic obstructive asthma with acute exacerbation"    
 [5] "Intrinsic asthma, with acute exacerbation"             
 [6] "Extrinsic asthma with acute exacerbation"              
 [7] "Obstructive chronic bronchitis with acute exacerbation"
 [8] "Mild persistent asthma with (acute) exacerbation"      
 [9] "Mild intermittent asthma with (acute) exacerbation"    
[10] "Moderate persistent asthma with (acute) exacerbation"

In [107]:
# select columns
exar.diag <- exar.diag %>% select(Subject_Id, Date, Diagnosis_Name, Code_Type, Code, 
                                  Diagnosis_Flag, Inpatient_Outpatient, Provider, 
                                  Hospital, Encounter_number) %>% arrange(Subject_Id, Date)
dim(exar.diag)

[1] 8601   10

## Acute exacerbation: Remove duplication

In [108]:
dim(exar.diag[duplicated(exar.diag), ]) # check duplication: 107

[1] 107  10

In [109]:
# remove duplicated rows
exar.diag.no.dup <- exar.diag[!duplicated(exar.diag), ] # remove 107
dim(exar.diag.no.dup)

[1] 8494   10

In [110]:
# some categories are same sub set -> change name
exar.diag.no.dup$Diagnosis_Name[exar.diag.no.dup$Diagnosis_Name == 'Chronic obstructive asthma with acute exacerbation'] <- 'Chronic obstructive pulmonary disease with (acute) exacerbation'
exar.diag.no.dup$Diagnosis_Name[exar.diag.no.dup$Diagnosis_Name == 'Obstructive chronic bronchitis with acute exacerbation'] <- 'Chronic obstructive pulmonary disease with (acute) exacerbation'
exar.diag.no.dup$Diagnosis_Name[exar.diag.no.dup$Diagnosis_Name == 'Extrinsic asthma with acute exacerbation'] <- 'Mild intermittent asthma with (acute) exacerbation'
exar.diag.no.dup$Diagnosis_Name[exar.diag.no.dup$Diagnosis_Name == 'Intrinsic asthma, with acute exacerbation'] <- 'Mild intermittent asthma with (acute) exacerbation'
exar.diag.no.dup$Diagnosis_Name[exar.diag.no.dup$Diagnosis_Name == 'Asthma, unspecified type, with acute exacerbation'] <- 'Unspecified asthma with (acute) exacerbation'
table(exar.diag.no.dup$Diagnosis_Name)


Chronic obstructive pulmonary disease with (acute) exacerbation 
                                                           1113 
             Mild intermittent asthma with (acute) exacerbation 
                                                           1539 
               Mild persistent asthma with (acute) exacerbation 
                                                            285 
           Moderate persistent asthma with (acute) exacerbation 
                                                           1164 
             Severe persistent asthma with (acute) exacerbation 
                                                            764 
                   Unspecified asthma with (acute) exacerbation 
                                                           3629 

In [111]:
dim(exar.diag.no.dup[duplicated(exar.diag.no.dup), ])
exar.diag.no.dup.1 <- exar.diag.no.dup[!duplicated(exar.diag.no.dup), ]
dim(exar.diag.no.dup.1[duplicated(exar.diag.no.dup.1), ])

[1]  0 10

[1]  0 10

In [112]:
# remove COPD
acute.exacer <- exar.diag.no.dup.1 %>% filter(Diagnosis_Name != 'Chronic obstructive pulmonary disease with (acute) exacerbation')
dim(acute.exacer)
table(acute.exacer$Diagnosis_Name)
length(unique(acute.exacer$Subject_Id)) # 483

[1] 7381   10


  Mild intermittent asthma with (acute) exacerbation 
                                                1539 
    Mild persistent asthma with (acute) exacerbation 
                                                 285 
Moderate persistent asthma with (acute) exacerbation 
                                                1164 
  Severe persistent asthma with (acute) exacerbation 
                                                 764 
        Unspecified asthma with (acute) exacerbation 
                                                3629 

[1] 483

In [113]:
dim(acute.exacer[duplicated(acute.exacer), ]) # 0

[1]  0 10

In [114]:
# export acute exacerbation diagnosis all
acute.exacer.diag.all <- acute.exacer %>% select(Subject_Id, Date, Diagnosis_Name) %>% arrange(Subject_Id, Date)
dim(acute.exacer.diag.all)

[1] 7381    3

In [115]:
# test duplication
dim(acute.exacer[duplicated(acute.exacer[,c('Subject_Id', 'Date')]), ]) # 3621

[1] 3621   10

**Different diagnosis on the same date. So prioritize based on severity**

In [116]:
# convert Cat_prior to acute exacerbation Category
table(acute.exacer$Diagnosis_Name)
acute.exacer$Cat_prior <- NA
acute.exacer$Cat_prior[acute.exacer$Diagnosis_Name == 'Unspecified asthma with (acute) exacerbation'] <- 0
acute.exacer$Cat_prior[acute.exacer$Diagnosis_Name == 'Mild intermittent asthma with (acute) exacerbation'] <- 1
acute.exacer$Cat_prior[acute.exacer$Diagnosis_Name == 'Mild persistent asthma with (acute) exacerbation'] <- 2
acute.exacer$Cat_prior[acute.exacer$Diagnosis_Name == 'Moderate persistent asthma with (acute) exacerbation'] <- 3
acute.exacer$Cat_prior[acute.exacer$Diagnosis_Name == 'Severe persistent asthma with (acute) exacerbation'] <- 4
table(acute.exacer$Cat_prior)


  Mild intermittent asthma with (acute) exacerbation 
                                                1539 
    Mild persistent asthma with (acute) exacerbation 
                                                 285 
Moderate persistent asthma with (acute) exacerbation 
                                                1164 
  Severe persistent asthma with (acute) exacerbation 
                                                 764 
        Unspecified asthma with (acute) exacerbation 
                                                3629 


   0    1    2    3    4 
3629 1539  285 1164  764 

In [117]:
# priority selecting patients with acute.exacer
acute.exacer.cat.prior <- acute.exacer %>%                                 # Get max by group
  group_by(Subject_Id, Date) %>%
  summarise_at(vars(Cat_prior),
               list(Cat_prior = max))
dim(acute.exacer.cat.prior)
# from 3911 to 3760

[1] 3760    3

In [118]:
dim(acute.exacer.cat.prior[duplicated(acute.exacer.cat.prior[,c('Subject_Id', 'Date')]), ]) # no dup

[1] 0 3

In [119]:
# convert Cat_prior to Asthma Category
table(acute.exacer.cat.prior$Cat_prior)
acute.exacer.cat.prior$Acute_Exacerbation_Category <- NA
acute.exacer.cat.prior$Acute_Exacerbation_Category[acute.exacer.cat.prior$Cat_prior == 0] <- 'Unspecified_asthma_with_acute_exacerbation'
acute.exacer.cat.prior$Acute_Exacerbation_Category[acute.exacer.cat.prior$Cat_prior == 1] <- 'Mild_intermittent_asthma_with_acute_exacerbation'
acute.exacer.cat.prior$Acute_Exacerbation_Category[acute.exacer.cat.prior$Cat_prior == 2] <- 'Mild_persistent_asthma_with_acute_exacerbation'
acute.exacer.cat.prior$Acute_Exacerbation_Category[acute.exacer.cat.prior$Cat_prior == 3] <- 'Moderate_persistent_asthma_with_acute_exacerbation'
acute.exacer.cat.prior$Acute_Exacerbation_Category[acute.exacer.cat.prior$Cat_prior == 4] <- 'Severe_persistent_asthma_with_acute_exacerbation'
table(acute.exacer.cat.prior$Acute_Exacerbation_Category)


   0    1    2    3    4 
1871  768  149  646  326 


  Mild_intermittent_asthma_with_acute_exacerbation 
                                               768 
    Mild_persistent_asthma_with_acute_exacerbation 
                                               149 
Moderate_persistent_asthma_with_acute_exacerbation 
                                               646 
  Severe_persistent_asthma_with_acute_exacerbation 
                                               326 
        Unspecified_asthma_with_acute_exacerbation 
                                              1871 

In [120]:
# export data with unique category for each date
acute.exacer.cat.prior <- acute.exacer.cat.prior %>% select(Subject_Id, Date, Acute_Exacerbation_Category)

In [121]:
# calculate total diagnosis
acute.exacer.cat.prior$Diagnosis <- 1 # assign 1 for diagnosis
dim(acute.exacer.cat.prior)

[1] 3760    4

In [122]:
# Group by Subject_Id and sum of diagnosis using dplyr
acute.exacer.counts.per.ind <- acute.exacer.cat.prior %>% group_by(Subject_Id) %>% 
                                                      summarise(Total_Acute_Exacerbation_Diagnosis = sum(Diagnosis),
                                                                .groups = 'drop')
dim(acute.exacer.counts.per.ind)

[1] 483   2

In [123]:
summary(acute.exacer.counts.per.ind$Total_Acute_Exacerbation_Diagnosis)
table(acute.exacer.counts.per.ind$Total_Acute_Exacerbation_Diagnosis)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
  1.000   1.000   3.000   7.785   8.000  87.000 


  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18  19  20 
138  69  45  32  24  18  21  16  10   9  10  11   9   4   5   6   2   4   3   4 
 21  22  23  24  25  26  28  29  30  31  32  34  36  37  38  42  47  52  60  67 
  3   4   4   1   3   1   3   1   1   1   1   2   2   1   1   2   1   2   1   1 
 74  76  81  82  84  87 
  2   1   1   1   1   1 

## Acute exacerbation: Most frequency diagnosis

In [124]:
# create dummy variables for each diag category in acute.exacer.cat.prior
acute.exacer.dummy <- dummy_cols(acute.exacer.cat.prior,
                                   select_columns = 'Acute_Exacerbation_Category')
dim(acute.exacer.dummy)

[1] 3760    9

In [125]:
# group by Subject_Id and sum all columns
acute.exacer.dummy.group <- acute.exacer.dummy[,-2:-4] # remove un-neccesary columns
sum.acute.exacer.diganosis <- acute.exacer.dummy.group %>% group_by(Subject_Id) %>% 
                                                    summarise(across(everything(), sum), .groups = 'drop') %>%
                                                    as.data.frame()
dim(sum.acute.exacer.diganosis)

[1] 483   6

In [126]:
# add column: Yes or No for each diagnosis
exist.sum.acute.exacer.diganosis <- sum.acute.exacer.diganosis
exist.sum.acute.exacer.diganosis[,-1] <- ifelse(exist.sum.acute.exacer.diganosis[,-1] > 0, 'Yes', 'No')
colnames(exist.sum.acute.exacer.diganosis)[-1] <- paste(colnames(exist.sum.acute.exacer.diganosis[,-1]),'_Existence_Yes_No', sep = '')

In [127]:
# add suffix count after each diagnosis
colnames(sum.acute.exacer.diganosis)<- paste(colnames(sum.acute.exacer.diganosis),'total_diagnosis',sep='_')
colnames(sum.acute.exacer.diganosis)[1] <- 'Subject_Id'

In [128]:
# merge
sum.acute.exacer.diganosis.final <- merge(sum.acute.exacer.diganosis, exist.sum.acute.exacer.diganosis, by = 'Subject_Id')
dim(sum.acute.exacer.diganosis.final)

[1] 483  11

In [129]:
# Remove the 'Acute_Exacerbation_Category' prefix from all column names
names(sum.acute.exacer.diganosis.final) <- gsub('Acute_Exacerbation_Category_', '', names(sum.acute.exacer.diganosis.final))
dim(sum.acute.exacer.diganosis.final)

[1] 483  11

In [130]:
# add column: Yes or No for diagnosis total
sum.acute.exacer.diganosis.final$Any_Acute_Exacerbation_Existence_Yes_No <- 'Yes' # yes for diagnosis

In [131]:
# find most frequency diagnosis
acute.exacer.cat.count <- sum.acute.exacer.diganosis.final[,c('Mild_intermittent_asthma_with_acute_exacerbation_total_diagnosis', 
                                       'Mild_persistent_asthma_with_acute_exacerbation_total_diagnosis',
                                       'Moderate_persistent_asthma_with_acute_exacerbation_total_diagnosis',
                                       'Severe_persistent_asthma_with_acute_exacerbation_total_diagnosis',
                                       'Unspecified_asthma_with_acute_exacerbation_total_diagnosis')]

sum.acute.exacer.diganosis.final$Most_freq_acute_exacerbarion_diagnosis <- colnames(acute.exacer.cat.count)[apply(acute.exacer.cat.count,1,which.max)]

# remove suffix _total_diagnosis
sum.acute.exacer.diganosis.final <- sum.acute.exacer.diganosis.final %>% mutate_at('Most_freq_acute_exacerbarion_diagnosis', str_replace, '_total_diagnosis', '')

## Acute exacerbation: Closest date before or after plasma collect date

In [132]:
# merge collect date with acute.exacer.cat.prior by Subject_ID
acute.exacer.cat.prior.1 <- merge(acute.exacer.cat.prior, 
                                  data.id[,c('Subject_Id', 'Plasma_collect_date')], 
                                  by = 'Subject_Id')
dim(acute.exacer.cat.prior.1)

[1] 3760    5

In [133]:
# convert the date column to the Y-M-D format
acute.exacer.cat.prior.1$Date <- as.Date(acute.exacer.cat.prior.1$Date, format = '%m/%d/%Y')
typeof(acute.exacer.cat.prior.1$Date)

# plasma collect date
acute.exacer.cat.prior.1$Plasma_collect_date <- as.Date(acute.exacer.cat.prior.1$Plasma_collect_date, format = '%Y -%m -%d')
typeof(acute.exacer.cat.prior.1$Plasma_collect_date)

[1] "double"

[1] "double"

In [134]:
# substract collect date and diag date
acute.exacer.cat.prior.1['Days_Difference'] <- difftime(acute.exacer.cat.prior.1$Plasma_collect_date, 
                                                        acute.exacer.cat.prior.1$Date, units = 'days')
head(acute.exacer.cat.prior.1$Days_Difference,50)
typeof(acute.exacer.cat.prior.1$Days_Difference)

# absolute values
acute.exacer.cat.prior.1['Days_Difference_Abs'] <- as.numeric(abs(acute.exacer.cat.prior.1$Days_Difference))
typeof(acute.exacer.cat.prior.1$Days_Difference_Abs)

Time differences in days
 [1] 2827 2826 2713 2664 2618 2443 2345 2343 2341 2338 2336 2327 2079 2078 2072
[16] 2069 2067 1625 1624 1432 1430 1428 1421 1381 1379 1344 1274 1249 1248 1244
[31] 1240 1238 1171 1170 1168 1167 1166 1165  795  604  603 -367 -368 -369 -493
[46] -636 -722 -894 -895 -896

[1] "double"

[1] "double"

In [135]:
dim(acute.exacer.cat.prior.1[duplicated(acute.exacer.cat.prior.1), ]) # no dup

[1] 0 7

In [136]:
# Find closest date to plasma collect date
acute.exacer.date.closest.collect <- acute.exacer.cat.prior.1 %>% select(Subject_Id, Date, 
                                                                         Days_Difference_Abs, 
                                                                         Acute_Exacerbation_Category)

In [137]:
# find closest date to plasma collect date 
acute.exacer.closest.collect.date <- acute.exacer.date.closest.collect %>%                     # Get min by group
  group_by(Subject_Id, Acute_Exacerbation_Category) %>%
  summarise_at(vars(Days_Difference_Abs),
               list(Closest_collect_date_gap = min))
dim(acute.exacer.closest.collect.date)

[1] 816   3

In [138]:
dim(acute.exacer.closest.collect.date[duplicated(acute.exacer.closest.collect.date), ]) # no dup

[1] 0 3

In [139]:
# group the data frame by 'Subject_Id' and spread the 'closet_collect_date' column
acute.exacer.closest.collect.date.wide <- acute.exacer.closest.collect.date %>% group_by(Subject_Id, 
                                                                                    Acute_Exacerbation_Category) %>%
                                                    pivot_wider(names_from = Acute_Exacerbation_Category, 
                                                                values_from = Closest_collect_date_gap)
dim(acute.exacer.closest.collect.date.wide)

[1] 483   6

In [140]:
# add suffix cloest collect date after each diag
colnames(acute.exacer.closest.collect.date.wide) <- paste(colnames(acute.exacer.closest.collect.date.wide),
                                                          'closest_collect_date_gap',sep='_')
colnames(acute.exacer.closest.collect.date.wide)[1] = 'Subject_Id'
dim(acute.exacer.closest.collect.date.wide)

[1] 483   6

In [141]:
# filter for all diagnosis to include diagnosis date for each days difference
acute.exacer.names <- unique(acute.exacer.date.closest.collect$Acute_Exacerbation_Category)
for (acute.exacer.name in acute.exacer.names){
    acute.exacer.date.closest.collect.each.diag <- acute.exacer.date.closest.collect %>% filter(Acute_Exacerbation_Category == acute.exacer.name)


    # remove diagnosis_name and Plasma_collect_date
    acute.exacer.date.closest.collect.each.diag.1 <- acute.exacer.date.closest.collect.each.diag %>% select(-c(Acute_Exacerbation_Category))


    # replace Date name with date + diagnosis
    names(acute.exacer.date.closest.collect.each.diag.1)[names(acute.exacer.date.closest.collect.each.diag.1) == 'Date'] <- paste(acute.exacer.name, '_closest_collect_date', sep = '')


    # test merge date for diagnosis
    acute.exacer.closest.collect.date.wide <- merge(acute.exacer.closest.collect.date.wide, acute.exacer.date.closest.collect.each.diag.1, 
                                                        by.x = c('Subject_Id', paste(acute.exacer.name, '_closest_collect_date_gap', sep = '')), 
                                                        by.y = c('Subject_Id', 'Days_Difference_Abs'), all.x = TRUE)
    }


dim(acute.exacer.closest.collect.date.wide)

[1] 483  11

In [142]:
# remove duplication
acute.exacer.check.unique.list <- c('Subject_Id',
                       'Mild_persistent_asthma_with_acute_exacerbation_closest_collect_date_gap',
                       'Severe_persistent_asthma_with_acute_exacerbation_closest_collect_date_gap',
                       'Moderate_persistent_asthma_with_acute_exacerbation_closest_collect_date_gap',
                       'Mild_intermittent_asthma_with_acute_exacerbation_closest_collect_date_gap',
                       'Unspecified_asthma_with_acute_exacerbation_closest_collect_date_gap')
acute.exacer.closest.collect.date.wide <- acute.exacer.closest.collect.date.wide[!duplicated(acute.exacer.closest.collect.date.wide[,acute.exacer.check.unique.list]), ]
dim(acute.exacer.closest.collect.date.wide)

[1] 483  11

In [143]:
# find closest asthma diagnosis to collect date
acute.exacer.closest.collect.date.cols <- acute.exacer.closest.collect.date.wide[,c('Mild_persistent_asthma_with_acute_exacerbation_closest_collect_date_gap',
                       'Severe_persistent_asthma_with_acute_exacerbation_closest_collect_date_gap',
                       'Moderate_persistent_asthma_with_acute_exacerbation_closest_collect_date_gap',
                       'Mild_intermittent_asthma_with_acute_exacerbation_closest_collect_date_gap',
                       'Unspecified_asthma_with_acute_exacerbation_closest_collect_date_gap')]


acute.exacer.closest.collect.date.wide$Closest_collect_date_acute_exacerbation_diagnosis <- colnames(acute.exacer.closest.collect.date.cols)[apply(data.matrix(acute.exacer.closest.collect.date.cols),1,which.min)]

# remove suffix _closest_collect_date_gap
acute.exacer.closest.collect.date.wide <- acute.exacer.closest.collect.date.wide %>% mutate_at('Closest_collect_date_acute_exacerbation_diagnosis', str_replace, '_closest_collect_date_gap', '')

In [144]:
# gap between closest date to collect date
min.na.rm <- function(x){
    min(x, na.rm = TRUE)
}

acute.exacer.closest.collect.date.wide$Closest_collect_date_acute_exacerbation_diagnosis_gap <- apply(data.matrix(acute.exacer.closest.collect.date.cols),1,min.na.rm)

## Acute exacerbation: Most recent diagnosis

In [145]:
typeof(acute.exacer.cat.prior$Date)

[1] "double"

In [146]:
# find recent date
recent.acute.exacer.date <- acute.exacer.cat.prior %>%                                   # Get max by group
  group_by(Subject_Id, Acute_Exacerbation_Category) %>%
  summarise_at(vars(Date),
               list(Recent_date = max))
dim(recent.acute.exacer.date)

[1] 816   3

In [147]:
# group the data frame by 'Subject_Id' and spread the 'max' column
recent.acute.exacer.date.wide <- recent.acute.exacer.date %>% group_by(Subject_Id, Acute_Exacerbation_Category) %>%
                                                    pivot_wider(names_from = Acute_Exacerbation_Category, 
                                                                values_from = Recent_date)
dim(recent.acute.exacer.date.wide)

[1] 483   6

In [148]:
# add suffix recent date after each diagnosis
colnames(recent.acute.exacer.date.wide) <- paste(colnames(recent.acute.exacer.date.wide),'recent_diagnosis_date',sep='_')
colnames(recent.acute.exacer.date.wide)[1] = 'Subject_Id'

In [149]:
# find recent acute.exacer diagnosis
# because recent date is string -> need to convert date formate
acute.exacer.recent.date.cols <- recent.acute.exacer.date.wide[,c('Mild_intermittent_asthma_with_acute_exacerbation_recent_diagnosis_date', 
                                                            'Mild_persistent_asthma_with_acute_exacerbation_recent_diagnosis_date',
                                                            'Moderate_persistent_asthma_with_acute_exacerbation_recent_diagnosis_date',
                                                            'Severe_persistent_asthma_with_acute_exacerbation_recent_diagnosis_date',
                                                            'Unspecified_asthma_with_acute_exacerbation_recent_diagnosis_date')]

convert.date <- function(x) as.Date(x, format = '%Y-%m-%d')
acute.exacer.recent.date.cols <- data.frame(lapply(acute.exacer.recent.date.cols, convert.date))
acute.exacer.recent.date.cols[is.na(acute.exacer.recent.date.cols)] <- as.Date('1900-01-01', format = '%Y-%m-%d')
recent.acute.exacer.date.wide$Most_recent_acute_exacerbation_diagnosis <- colnames(acute.exacer.recent.date.cols)[apply(data.matrix(acute.exacer.recent.date.cols),1,which.max)]

# remove suffix _recent_diagnosis_date
recent.acute.exacer.date.wide <- recent.acute.exacer.date.wide %>% mutate_at('Most_recent_acute_exacerbation_diagnosis', str_replace, '_recent_diagnosis_date', '')

## Acute exacerbation: First date

In [150]:
# find first date in acute.exacer.cat.prior
first.acute.exacer.date <- acute.exacer.cat.prior  %>%                                      # Get min by group
  group_by(Subject_Id, Acute_Exacerbation_Category) %>%
  summarise_at(vars(Date),
               list(First_date = min))
dim(first.acute.exacer.date)

[1] 816   3

In [151]:
# group the data frame by 'Subject_Id' and spread the 'min' column
first.acute.exacer.date.wide <- first.acute.exacer.date %>% group_by(Subject_Id, Acute_Exacerbation_Category) %>%
                                                    pivot_wider(names_from = Acute_Exacerbation_Category, 
                                                                values_from = First_date)
dim(first.acute.exacer.date.wide)

[1] 483   6

In [152]:
# add suffix first date after each diagnosis
colnames(first.acute.exacer.date.wide) <- paste(colnames(first.acute.exacer.date.wide),'first_diagnosis_date',sep='_')
colnames(first.acute.exacer.date.wide)[1] = 'Subject_Id'

In [153]:
dim(first.acute.exacer.date.wide)
length(unique(first.acute.exacer.date.wide$Subject_Id)) # 483

[1] 483   6

[1] 483

In [154]:
# merge files
dim(acute.exacer.closest.collect.date.wide)

dim(recent.acute.exacer.date.wide)

dim(first.acute.exacer.date.wide)

dim(sum.acute.exacer.diganosis.final)

[1] 483  13

[1] 483   7

[1] 483   6

[1] 483  13

In [155]:
acute.exacer.final <- sum.acute.exacer.diganosis.final %>% left_join(acute.exacer.counts.per.ind, by = 'Subject_Id') %>%
                                             left_join(acute.exacer.closest.collect.date.wide, by = 'Subject_Id') %>%
                                             left_join(recent.acute.exacer.date.wide, by = 'Subject_Id') %>%
                                             left_join(first.acute.exacer.date.wide, by = 'Subject_Id')
dim(acute.exacer.final)

[1] 483  37

In [156]:
acute.exacer.final <- acute.exacer.final %>% select(Subject_Id,
                                            Any_Acute_Exacerbation_Existence_Yes_No,
                                            Total_Acute_Exacerbation_Diagnosis,
                                            Most_freq_acute_exacerbarion_diagnosis,
                                            Most_recent_acute_exacerbation_diagnosis,
                                            Closest_collect_date_acute_exacerbation_diagnosis,
                                            Closest_collect_date_acute_exacerbation_diagnosis_gap,
                                            
                                            Mild_intermittent_asthma_with_acute_exacerbation_Existence_Yes_No,
                                            Mild_intermittent_asthma_with_acute_exacerbation_total_diagnosis,
                                            
                                            Mild_intermittent_asthma_with_acute_exacerbation_first_diagnosis_date,
                                            Mild_intermittent_asthma_with_acute_exacerbation_recent_diagnosis_date,
                                            Mild_intermittent_asthma_with_acute_exacerbation_closest_collect_date,
                                            Mild_intermittent_asthma_with_acute_exacerbation_closest_collect_date_gap,
                                            Mild_persistent_asthma_with_acute_exacerbation_Existence_Yes_No,
                                            Mild_persistent_asthma_with_acute_exacerbation_total_diagnosis,
                                            
                                            Mild_persistent_asthma_with_acute_exacerbation_first_diagnosis_date,
                                            Mild_persistent_asthma_with_acute_exacerbation_recent_diagnosis_date,
                                            Mild_persistent_asthma_with_acute_exacerbation_closest_collect_date,
                                            Mild_persistent_asthma_with_acute_exacerbation_closest_collect_date_gap,
                                            Moderate_persistent_asthma_with_acute_exacerbation_Existence_Yes_No,
                                            Moderate_persistent_asthma_with_acute_exacerbation_total_diagnosis,
                                            
                                            Moderate_persistent_asthma_with_acute_exacerbation_first_diagnosis_date,
                                            Moderate_persistent_asthma_with_acute_exacerbation_recent_diagnosis_date,
                                            Moderate_persistent_asthma_with_acute_exacerbation_closest_collect_date,
                                            Moderate_persistent_asthma_with_acute_exacerbation_closest_collect_date_gap,
                                            Severe_persistent_asthma_with_acute_exacerbation_Existence_Yes_No,
                                            Severe_persistent_asthma_with_acute_exacerbation_total_diagnosis,
                                            
                                            Severe_persistent_asthma_with_acute_exacerbation_first_diagnosis_date,
                                            Severe_persistent_asthma_with_acute_exacerbation_recent_diagnosis_date,
                                            Severe_persistent_asthma_with_acute_exacerbation_closest_collect_date,
                                            Severe_persistent_asthma_with_acute_exacerbation_closest_collect_date_gap,
                                            Unspecified_asthma_with_acute_exacerbation_Existence_Yes_No,
                                            Unspecified_asthma_with_acute_exacerbation_total_diagnosis,
                                            
                                            Unspecified_asthma_with_acute_exacerbation_first_diagnosis_date,
                                            Unspecified_asthma_with_acute_exacerbation_recent_diagnosis_date,
                                            Unspecified_asthma_with_acute_exacerbation_closest_collect_date,
                                            Unspecified_asthma_with_acute_exacerbation_closest_collect_date_gap)
dim(acute.exacer.final)

[1] 483  37

# Adrenal insufficiency

In [157]:
# adrenal insufficiency
adr.insuff <- c('Corticoadrenal insufficiency', 'Other adrenocortical insufficiency', 
                'Primary adrenocortical insufficiency', 'Unspecified adrenocortical insufficiency')

In [158]:
# filter interested diagnosis based on biobank folder
adr.insuff.diag <- filter(diag.data.ID, Diagnosis_Name %in% adr.insuff) 
dim(adr.insuff.diag)

[1] 5054   15

In [159]:
length(unique(adr.insuff.diag$Diagnosis_Name)) # 4 different diagnosis
table(adr.insuff.diag$Diagnosis_Name)
table(adr.insuff.diag$Code_Type)
table(adr.insuff.diag$Code)

[1] 4


            Corticoadrenal insufficiency 
                                     276 
      Other adrenocortical insufficiency 
                                     841 
    Primary adrenocortical insufficiency 
                                     809 
Unspecified adrenocortical insufficiency 
                                    3128 


ICD10  ICD9 
 4778   276 


 255.4  E27.1 E27.40 E27.49 
   276    809   3128    841 

In [160]:
adr.insuff.diag <- adr.insuff.diag %>% select(Subject_Id, Date, Diagnosis_Name, Code_Type, Code,  
                                  Diagnosis_Flag, Inpatient_Outpatient, Provider, 
                                  Hospital, Encounter_number)
dim(adr.insuff.diag)

[1] 5054   10

In [161]:
dim(adr.insuff.diag[duplicated(adr.insuff.diag), ]) # 101

[1] 101  10

In [162]:
# remove duplicated rows
adr.insuff.diag.no.dup <- adr.insuff.diag[!duplicated(adr.insuff.diag), ]
dim(adr.insuff.diag.no.dup)

[1] 4953   10

In [163]:
dim(adr.insuff.diag.no.dup[duplicated(adr.insuff.diag.no.dup), ]) # no dup

[1]  0 10

In [164]:
# export adr.insuff. diagnosis all
adr.insuff.diag.all <- adr.insuff.diag.no.dup %>% select(Subject_Id, Date, Diagnosis_Name) %>% arrange(Subject_Id, Date)

In [165]:
# test duplication before remove
dim(adr.insuff.diag.no.dup[duplicated(adr.insuff.diag.no.dup[,c('Subject_Id', 'Date')]), ]) # 2094

[1] 2094   10

In [166]:
# convert Cat_prior to adrenal insufficiency Category
table(adr.insuff.diag.no.dup$Diagnosis_Name)
adr.insuff.diag.no.dup$Cat_prior <- NA

adr.insuff.diag.no.dup$Cat_prior[adr.insuff.diag.no.dup$Diagnosis_Name == 'Unspecified adrenocortical insufficiency'] <- 0
adr.insuff.diag.no.dup$Cat_prior[adr.insuff.diag.no.dup$Diagnosis_Name == 'Other adrenocortical insufficiency'] <- 1
adr.insuff.diag.no.dup$Cat_prior[adr.insuff.diag.no.dup$Diagnosis_Name == 'Corticoadrenal insufficiency'] <- 2
adr.insuff.diag.no.dup$Cat_prior[adr.insuff.diag.no.dup$Diagnosis_Name == 'Primary adrenocortical insufficiency'] <- 3
table(adr.insuff.diag.no.dup$Cat_prior)


            Corticoadrenal insufficiency 
                                     276 
      Other adrenocortical insufficiency 
                                     803 
    Primary adrenocortical insufficiency 
                                     806 
Unspecified adrenocortical insufficiency 
                                    3068 


   0    1    2    3 
3068  803  276  806 

In [167]:
# priority selecting patients with adr.insuff.diag.no.dup
adr.insuff.cat.prior <- adr.insuff.diag.no.dup %>%                                 # Get max by group
  group_by(Subject_Id, Date) %>%
  summarise_at(vars(Cat_prior),
               list(Cat_prior = max))
dim(adr.insuff.cat.prior)
# from 2954 to 2859

[1] 2859    3

In [168]:
dim(adr.insuff.cat.prior[duplicated(adr.insuff.cat.prior[,c('Subject_Id', 'Date')]), ]) # no dup

[1] 0 3

In [169]:
# convert Cat_prior to Adrenal_Insufficiency Category
table(adr.insuff.cat.prior$Cat_prior)
adr.insuff.cat.prior$Adrenal_Insufficiency_Category <- NA
adr.insuff.cat.prior$Adrenal_Insufficiency_Category[adr.insuff.cat.prior$Cat_prior == 0] <- 'Unspecified_adrenocortical_insufficiency'
adr.insuff.cat.prior$Adrenal_Insufficiency_Category[adr.insuff.cat.prior$Cat_prior == 1] <- 'Other_adrenocortical_insufficiency'
adr.insuff.cat.prior$Adrenal_Insufficiency_Category[adr.insuff.cat.prior$Cat_prior == 2] <- 'Corticoadrenal_insufficiency'
adr.insuff.cat.prior$Adrenal_Insufficiency_Category[adr.insuff.cat.prior$Cat_prior == 3] <- 'Primary_adrenocortical_insufficiency'

table(adr.insuff.cat.prior$Adrenal_Insufficiency_Category)


   0    1    2    3 
1771  404  216  468 


            Corticoadrenal_insufficiency 
                                     216 
      Other_adrenocortical_insufficiency 
                                     404 
    Primary_adrenocortical_insufficiency 
                                     468 
Unspecified_adrenocortical_insufficiency 
                                    1771 

In [170]:
# export data with unique category for each date
adr.insuff.cat.prior <- adr.insuff.cat.prior %>% select(Subject_Id, Date, Adrenal_Insufficiency_Category) %>%
                                                 arrange(Subject_Id, Date)

In [171]:
# calculate for total diagnosis
adr.insuff.cat.prior$Diagnosis <- 1 # assign 1 for diagnosis

In [172]:
# Group by Subject_Id and sum of diagnosis using dplyr
adr.insuff.counts.per.ind <- adr.insuff.cat.prior %>% group_by(Subject_Id) %>% 
  summarise(Total_Adrenal_Insufficiency_Diagnosis = sum(Diagnosis),
            .groups = 'drop')
dim(adr.insuff.counts.per.ind)

[1] 171   2

In [173]:
summary(adr.insuff.counts.per.ind$Total_Adrenal_Insufficiency_Diagnosis)
table(adr.insuff.counts.per.ind$Total_Adrenal_Insufficiency_Diagnosis)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
   1.00    2.00    4.00   16.72   16.00  608.00 


  1   2   3   4   5   6   7   8   9  11  12  13  15  16  18  19  20  21  22  23 
 41  21  13  11  14   4   2   8   1   2   3   5   2   2   4   1   3   2   4   3 
 24  25  26  27  28  29  33  34  35  36  46  51  53  54  55  66  81  99 121 319 
  2   1   1   1   1   1   2   1   3   1   1   1   1   1   1   1   1   1   1   1 
608 
  1 

## Adrenal insufficiency: Most frequency diagnosis

In [174]:
# create dummy variables for each diag category in adr.insuff.cat.prior
adr.insuff.dummy <- dummy_cols(adr.insuff.cat.prior,
                   select_columns = 'Adrenal_Insufficiency_Category')
dim(adr.insuff.dummy)

[1] 2859    8

In [175]:
# group by Subject_Id and sum all columns
adr.insuff.dummy.group <- adr.insuff.dummy[,-2:-4] # remove un-neccesary columns
sum.adr.insuff.diganosis <- adr.insuff.dummy.group %>% group_by(Subject_Id) %>% 
                                                    summarise(across(everything(), sum), .groups = 'drop') %>%
                                                    as.data.frame()
dim(sum.adr.insuff.diganosis)

[1] 171   5

In [176]:
# add column: Yes or No for each diagnosis
exist.sum.adr.insuff.diganosis <- sum.adr.insuff.diganosis
exist.sum.adr.insuff.diganosis[,-1] <- ifelse(exist.sum.adr.insuff.diganosis[,-1] > 0, 'Yes', 'No')
colnames(exist.sum.adr.insuff.diganosis)[-1] <- paste(colnames(exist.sum.adr.insuff.diganosis[,-1]),'_Existence_Yes_No', sep = '')

In [177]:
# add suffix count after each diagnosis
colnames(sum.adr.insuff.diganosis)<- paste(colnames(sum.adr.insuff.diganosis),'total_diagnosis',sep='_')
colnames(sum.adr.insuff.diganosis)[1] <- 'Subject_Id'

In [178]:
# merge
sum.adr.insuff.diganosis.final <- merge(sum.adr.insuff.diganosis, exist.sum.adr.insuff.diganosis, by = 'Subject_Id')
dim(sum.adr.insuff.diganosis.final)

[1] 171   9

In [179]:
# Remove the 'Diagnosis_Name_' prefix from all column names
names(sum.adr.insuff.diganosis.final) <- gsub('Adrenal_Insufficiency_Category_', '', names(sum.adr.insuff.diganosis.final))
dim(sum.adr.insuff.diganosis.final)

[1] 171   9

In [180]:
# add column: Yes or No for diagnosis total
sum.adr.insuff.diganosis.final$Any_Adrenal_Insufficiency_Existence_Yes_No <- 'Yes' # yes for diagnosis

In [181]:
# find most frequency diagnosis
adr.insuff.cat.count <- sum.adr.insuff.diganosis.final[,c('Corticoadrenal_insufficiency_total_diagnosis', 
                                       'Other_adrenocortical_insufficiency_total_diagnosis',
                                       'Primary_adrenocortical_insufficiency_total_diagnosis',
                                       'Unspecified_adrenocortical_insufficiency_total_diagnosis')]

sum.adr.insuff.diganosis.final$Most_freq_adrenal_insufficiency_diagnosis <- colnames(adr.insuff.cat.count)[apply(adr.insuff.cat.count,1,which.max)]

# remove suffix _total_diagnosis
sum.adr.insuff.diganosis.final <- sum.adr.insuff.diganosis.final %>% mutate_at('Most_freq_adrenal_insufficiency_diagnosis', str_replace, '_total_diagnosis', '')

## Adrenal insufficiency: Most recent diagnosis

In [182]:
typeof(adr.insuff.cat.prior$Date)

[1] "double"

In [183]:
# find recent date
recent.adr.insuff.date <- adr.insuff.cat.prior %>%                                       # Get max by group
  group_by(Subject_Id, Adrenal_Insufficiency_Category) %>%
  summarise_at(vars(Date),
               list(Recent_date = max))
dim(recent.adr.insuff.date)

[1] 255   3

In [184]:
# group the data frame by 'Subject_Id' and spread the 'max' column
recent.adr.insuff.date.wide <- recent.adr.insuff.date %>% group_by(Subject_Id, Adrenal_Insufficiency_Category) %>%
                                                    pivot_wider(names_from = Adrenal_Insufficiency_Category, 
                                                                values_from = Recent_date)
dim(recent.adr.insuff.date.wide)

[1] 171   5

In [185]:
# group the data frame by 'Subject_Id' and spread the 'max' column
recent.adr.insuff.date.wide <- recent.adr.insuff.date %>% group_by(Subject_Id, Adrenal_Insufficiency_Category) %>%
                                                    pivot_wider(names_from = Adrenal_Insufficiency_Category, 
                                                                values_from = Recent_date)
dim(recent.adr.insuff.date.wide)

[1] 171   5

In [186]:
# add suffix recent date after each diagnosis
colnames(recent.adr.insuff.date.wide) <- paste(colnames(recent.adr.insuff.date.wide),'recent_diagnosis_date',sep='_')
colnames(recent.adr.insuff.date.wide)[1] = 'Subject_Id'

In [187]:
# find recent adr.insuff diagnosis
# because recent date is string -> need to convert date formate
adr.insuff.recent.date.cols <- recent.adr.insuff.date.wide[,c('Corticoadrenal_insufficiency_recent_diagnosis_date', 
                                                            'Unspecified_adrenocortical_insufficiency_recent_diagnosis_date',
                                                            'Other_adrenocortical_insufficiency_recent_diagnosis_date',
                                                            'Primary_adrenocortical_insufficiency_recent_diagnosis_date')]

convert.date <- function(x) as.Date(x, format = '%Y-%m-%d')
adr.insuff.recent.date.cols <- data.frame(lapply(adr.insuff.recent.date.cols, convert.date))
adr.insuff.recent.date.cols[is.na(adr.insuff.recent.date.cols)] <- as.Date('1900-01-01', format = '%Y-%m-%d')
recent.adr.insuff.date.wide$Most_recent_adrenal_insufficiency_diagnosis <- colnames(adr.insuff.recent.date.cols)[apply(data.matrix(adr.insuff.recent.date.cols),1,which.max)]

## Adrenal insufficiency: Closest date before or after plasma collect date

In [189]:
# merge collect date with adr.insuff.cat.prior by Subject_ID
adr.insuff.cat.prior.1 <- merge(adr.insuff.cat.prior, data.id[,c('Subject_Id', 'Plasma_collect_date')], 
                                by = 'Subject_Id')
dim(adr.insuff.cat.prior.1)

[1] 2859    5

In [190]:
# convert the date column to the Y-M-D format
adr.insuff.cat.prior.1$Date <- as.Date(adr.insuff.cat.prior.1$Date, format = '%m/%d/%Y')
typeof(adr.insuff.cat.prior.1$Date)
# plasma collect date
adr.insuff.cat.prior.1$Plasma_collect_date <- as.Date(adr.insuff.cat.prior.1$Plasma_collect_date, format = '%Y -%m -%d')

[1] "double"

In [191]:
# substract collect date and diag date
adr.insuff.cat.prior.1['Days_Difference'] <- difftime(adr.insuff.cat.prior.1$Plasma_collect_date, 
                                                      adr.insuff.cat.prior.1$Date, units = 'days')


# absolute values
adr.insuff.cat.prior.1['Days_Difference_Abs'] <- as.numeric(abs(adr.insuff.cat.prior.1$Days_Difference))

In [192]:
dim(adr.insuff.cat.prior.1[duplicated(adr.insuff.cat.prior.1), ]) # no dup

[1] 0 7

In [193]:
# Find closest date to plasma collect date
adr.insuff.date.closest.collect <- adr.insuff.cat.prior.1 %>% select(Subject_Id, Date, 
                                                                     Days_Difference_Abs, 
                                                                     Adrenal_Insufficiency_Category)

In [194]:
# find closest date to plasma collect date in adr.insuff.date.closest.collect
adr.insuff.closest.collect.date <- adr.insuff.date.closest.collect %>%                        # Get min by group
  group_by(Subject_Id, Adrenal_Insufficiency_Category) %>%
  summarise_at(vars(Days_Difference_Abs),
               list(Closest_collect_date_gap = min))
dim(adr.insuff.closest.collect.date)

[1] 255   3

In [195]:
dim(adr.insuff.closest.collect.date[duplicated(adr.insuff.closest.collect.date), ]) # no dup

[1] 0 3

In [196]:
# group the data frame by 'Subject_Id' and spread the 'closet_collect_date' column
adr.insuff.closest.collect.date.wide <- adr.insuff.closest.collect.date %>% group_by(Subject_Id, 
                                                                                     Adrenal_Insufficiency_Category) %>%
                                                    pivot_wider(names_from = Adrenal_Insufficiency_Category, 
                                                                values_from = Closest_collect_date_gap)
dim(adr.insuff.closest.collect.date.wide)

[1] 171   5

In [197]:
# add suffix cloest collect date after each diag
colnames(adr.insuff.closest.collect.date.wide) <- paste(colnames(adr.insuff.closest.collect.date.wide),
                                                        'closest_collect_date_gap',sep='_')
colnames(adr.insuff.closest.collect.date.wide)[1] = 'Subject_Id'
dim(adr.insuff.closest.collect.date.wide)

[1] 171   5

In [198]:
# filter for all diagnosis to include diagnosis date for each days difference
adr.insuff.names <- unique(adr.insuff.date.closest.collect$Adrenal_Insufficiency_Category)
for (adr.insuff.name in adr.insuff.names){
    adr.insuff.date.closest.collect.each.diag <- adr.insuff.date.closest.collect %>% filter(Adrenal_Insufficiency_Category == adr.insuff.name)


    # remove diagnosis_name and Plasma_collect_date
    adr.insuff.date.closest.collect.each.diag.1 <- adr.insuff.date.closest.collect.each.diag %>% select(-c(Adrenal_Insufficiency_Category))


    # replace Date name with date + diagnosis
    names(adr.insuff.date.closest.collect.each.diag.1)[names(adr.insuff.date.closest.collect.each.diag.1) == 'Date'] <- paste(adr.insuff.name, '_closest_collect_date', sep = '')


    # test merge date for diagnosis
    adr.insuff.closest.collect.date.wide <- merge(adr.insuff.closest.collect.date.wide, adr.insuff.date.closest.collect.each.diag.1, 
                                                        by.x = c('Subject_Id', paste(adr.insuff.name, '_closest_collect_date_gap', sep = '')), 
                                                        by.y = c('Subject_Id', 'Days_Difference_Abs'), all.x = TRUE)
    }


dim(adr.insuff.closest.collect.date.wide)

[1] 171   9

In [199]:
# remove duplication
adr.insuff.check.unique.list <- c('Subject_Id',
                       'Primary_adrenocortical_insufficiency_closest_collect_date_gap',
                       'Other_adrenocortical_insufficiency_closest_collect_date_gap',
                       'Unspecified_adrenocortical_insufficiency_closest_collect_date_gap',
                       'Corticoadrenal_insufficiency_closest_collect_date_gap')
adr.insuff.closest.collect.date.wide <- adr.insuff.closest.collect.date.wide[!duplicated(adr.insuff.closest.collect.date.wide[,adr.insuff.check.unique.list]), ]
dim(adr.insuff.closest.collect.date.wide)

[1] 171   9

In [200]:
# find closest adr.insuff diagnosis to collect date
adr.insuff.closest.collect.date.cols <- adr.insuff.closest.collect.date.wide[,c('Primary_adrenocortical_insufficiency_closest_collect_date_gap',
                       'Other_adrenocortical_insufficiency_closest_collect_date_gap',
                       'Unspecified_adrenocortical_insufficiency_closest_collect_date_gap',
                       'Corticoadrenal_insufficiency_closest_collect_date_gap')]


adr.insuff.closest.collect.date.wide$Closest_collect_date_adrenal_insufficiency_diagnosis <- colnames(adr.insuff.closest.collect.date.cols)[apply(data.matrix(adr.insuff.closest.collect.date.cols),1,which.min)]

# remove suffix _closest_collect_date_gap
adr.insuff.closest.collect.date.wide <- adr.insuff.closest.collect.date.wide %>% mutate_at('Closest_collect_date_adrenal_insufficiency_diagnosis', str_replace, '_closest_collect_date_gap', '')

In [201]:
# gap between closest date to collect date
min.na.rm <- function(x){
    min(x, na.rm = TRUE)
}

adr.insuff.closest.collect.date.wide$Closest_collect_date_adrenal_insufficiency_diagnosis_gap <- apply(data.matrix(adr.insuff.closest.collect.date.cols),1,min.na.rm)

## Adrenal insufficiency: First date

In [202]:
# find first date in adr.insuff.cat.prior
first.adr.insuff.date <- adr.insuff.cat.prior  %>%                                      # Get min by group
  group_by(Subject_Id, Adrenal_Insufficiency_Category) %>%
  summarise_at(vars(Date),
               list(First_date = min))
dim(first.adr.insuff.date)

[1] 255   3

In [203]:
# group the data frame by 'Subject_Id' and spread the 'min' column
first.adr.insuff.date.wide <- first.adr.insuff.date %>% group_by(Subject_Id, Adrenal_Insufficiency_Category) %>%
                                                    pivot_wider(names_from = Adrenal_Insufficiency_Category, 
                                                                values_from = First_date)
dim(first.adr.insuff.date.wide)

[1] 171   5

In [204]:
# add suffix first date after each diagnosis
colnames(first.adr.insuff.date.wide) <- paste(colnames(first.adr.insuff.date.wide),'first_diagnosis_date',sep='_')
colnames(first.adr.insuff.date.wide)[1] = 'Subject_Id'

In [205]:
dim(first.adr.insuff.date.wide)
length(unique(first.adr.insuff.date.wide$Subject_Id)) # 171

[1] 171   5

[1] 171

In [206]:
# merge files
dim(adr.insuff.closest.collect.date.wide)

dim(recent.adr.insuff.date.wide)

dim(first.adr.insuff.date.wide)

dim(sum.adr.insuff.diganosis.final)

[1] 171  11

[1] 171   6

[1] 171   5

[1] 171  11

In [207]:
adr.insuff.final <- sum.adr.insuff.diganosis.final %>% left_join(adr.insuff.counts.per.ind, by = 'Subject_Id') %>%
                                             left_join(adr.insuff.closest.collect.date.wide, by = 'Subject_Id') %>%
                                             left_join(recent.adr.insuff.date.wide, by = 'Subject_Id') %>%
                                             left_join(first.adr.insuff.date.wide, by = 'Subject_Id')
                                            
dim(adr.insuff.final)

[1] 171  31

In [208]:
adr.insuff.final<- adr.insuff.final %>% select(Subject_Id,
                                                    Any_Adrenal_Insufficiency_Existence_Yes_No,
                                                    Total_Adrenal_Insufficiency_Diagnosis,
                                                    Most_freq_adrenal_insufficiency_diagnosis,
                                                    Most_recent_adrenal_insufficiency_diagnosis,
                                                    Closest_collect_date_adrenal_insufficiency_diagnosis,
                                                    Closest_collect_date_adrenal_insufficiency_diagnosis_gap,
                                                    
                                                    Corticoadrenal_insufficiency_Existence_Yes_No,
                                                    Corticoadrenal_insufficiency_total_diagnosis,
                                                    
                                                    Corticoadrenal_insufficiency_first_diagnosis_date,
                                                    Corticoadrenal_insufficiency_recent_diagnosis_date,
                                                    Corticoadrenal_insufficiency_closest_collect_date,
                                                    Corticoadrenal_insufficiency_closest_collect_date_gap,
                                                    Primary_adrenocortical_insufficiency_Existence_Yes_No,
                                                    Primary_adrenocortical_insufficiency_total_diagnosis,
                                                    
                                                    Primary_adrenocortical_insufficiency_first_diagnosis_date,
                                                    Primary_adrenocortical_insufficiency_recent_diagnosis_date,
                                                    Primary_adrenocortical_insufficiency_closest_collect_date,
                                                    Primary_adrenocortical_insufficiency_closest_collect_date_gap,
                                                    Unspecified_adrenocortical_insufficiency_Existence_Yes_No,
                                                    Unspecified_adrenocortical_insufficiency_total_diagnosis,
                                                    
                                                    Unspecified_adrenocortical_insufficiency_first_diagnosis_date,
                                                    Unspecified_adrenocortical_insufficiency_recent_diagnosis_date,
                                                    Unspecified_adrenocortical_insufficiency_closest_collect_date,
                                                    Unspecified_adrenocortical_insufficiency_closest_collect_date_gap,
                                                    Other_adrenocortical_insufficiency_Existence_Yes_No,
                                                    Other_adrenocortical_insufficiency_total_diagnosis,
                                                    
                                                    Other_adrenocortical_insufficiency_first_diagnosis_date,
                                                    Other_adrenocortical_insufficiency_recent_diagnosis_date,
                                                    Other_adrenocortical_insufficiency_closest_collect_date,
                                                    Other_adrenocortical_insufficiency_closest_collect_date_gap)
dim(adr.insuff.final)

[1] 171  31

# COPD

In [210]:
# load COPD file
copd.data <- read.csv(file.path(raw.rpdr.dir, 'COPD_PPV_Biobank.csv'))
dim(copd.data)
length(unique(copd.data$Subject_Id)) # only 927 indviduals

copd.data <- copd.data %>% select(Subject_Id, COPD_current_or_past_history_custom_PPV_greater_0.80PPV_Existence_Yes_No,
                                    COPD_current_or_past_history_custom_PPV_greater_0.90PPV_Existence_Yes_No)

[1] 927   8

[1] 927

# Merge all files

In [211]:
data.diag <- asthma.final %>% left_join(bronc.final, by = 'Subject_Id') %>% 
                                    left_join(copd.data, by = 'Subject_Id') %>%
                                    left_join(bronchtitis.final, by = 'Subject_Id') %>%
                                    left_join(acute.exacer.final, by = 'Subject_Id') %>% 
                                    left_join(adr.insuff.final, by = 'Subject_Id')
dim(data.diag)

[1] 928 109

In [212]:
diag.existence.cols <- colnames(data.diag %>% select(matches('Existence_Yes_No')))
# summary statistic of diagnosis existence
for (i in c(1:length(diag.existence.cols))){
    print(diag.existence.cols[i])
    print(table(data.diag[,diag.existence.cols[i]]))
}

[1] "Any_Asthma_Diagnosis_Existence_Yes_No"

Yes 
928 
[1] "Mild_intermittent_asthma_Existence_Yes_No"

 No Yes 
274 654 
[1] "Mild_persistent_asthma_Existence_Yes_No"

 No Yes 
698 230 
[1] "Moderate_persistent_asthma_Existence_Yes_No"

 No Yes 
584 344 
[1] "Severe_persistent_asthma_Existence_Yes_No"

 No Yes 
803 125 
[1] "Other_and_unspecificed_asthma_Existence_Yes_No"

 No Yes 
 13 915 
[1] "Any_Bronchiectasis_Existence_Yes_No"

Yes 
194 
[1] "COPD_current_or_past_history_custom_PPV_greater_0.80PPV_Existence_Yes_No"

 No Yes 
845  82 
[1] "COPD_current_or_past_history_custom_PPV_greater_0.90PPV_Existence_Yes_No"

 No Yes 
883  44 
[1] "Any_Chronic_Bronchitis_Existence_Yes_No"

Yes 
196 
[1] "Any_Acute_Exacerbation_Existence_Yes_No"

Yes 
483 
[1] "Mild_intermittent_asthma_with_acute_exacerbation_Existence_Yes_No"

 No Yes 
286 197 
[1] "Mild_persistent_asthma_with_acute_exacerbation_Existence_Yes_No"

 No Yes 
430  53 
[1] "Moderate_persistent_asthma_with_acute_exacerbation_Existe

In [213]:
# change empty value in existence columns to NO
for (i in c(1:length(diag.existence.cols))){
    data.diag[,diag.existence.cols[i]][is.na(data.diag[,diag.existence.cols[i]])] <- 'No'
}

In [214]:
# check table of all existence columns
for (i in c(1:length(diag.existence.cols))){
    print(diag.existence.cols[i])
    print(table(data.diag[,diag.existence.cols[i]]))
}

[1] "Any_Asthma_Diagnosis_Existence_Yes_No"

Yes 
928 
[1] "Mild_intermittent_asthma_Existence_Yes_No"

 No Yes 
274 654 
[1] "Mild_persistent_asthma_Existence_Yes_No"

 No Yes 
698 230 
[1] "Moderate_persistent_asthma_Existence_Yes_No"

 No Yes 
584 344 
[1] "Severe_persistent_asthma_Existence_Yes_No"

 No Yes 
803 125 
[1] "Other_and_unspecificed_asthma_Existence_Yes_No"

 No Yes 
 13 915 
[1] "Any_Bronchiectasis_Existence_Yes_No"

 No Yes 
734 194 
[1] "COPD_current_or_past_history_custom_PPV_greater_0.80PPV_Existence_Yes_No"

 No Yes 
846  82 
[1] "COPD_current_or_past_history_custom_PPV_greater_0.90PPV_Existence_Yes_No"

 No Yes 
884  44 
[1] "Any_Chronic_Bronchitis_Existence_Yes_No"

 No Yes 
732 196 
[1] "Any_Acute_Exacerbation_Existence_Yes_No"

 No Yes 
445 483 
[1] "Mild_intermittent_asthma_with_acute_exacerbation_Existence_Yes_No"

 No Yes 
731 197 
[1] "Mild_persistent_asthma_with_acute_exacerbation_Existence_Yes_No"

 No Yes 
875  53 
[1] "Moderate_persistent_asthma_with_a